<a href="https://colab.research.google.com/github/AndresLAraque/Sistemas-Recomendacion/blob/main/DBpedia_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Modelo Colaborativo

In [2]:
#!pip install scikit-learn
#!pip install pandas
#!pip install seaborn
#!pip install numpy
#!pip install scikit-surprise

In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import json
import pandas as pd
#genometags=pd.read_csv('/home/andres/Documents/Jupyter/Enrriquecimient Semantic/ml-latest/genome-tags.csv')
movies=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SistemasRecomendacion/Talleres/Taller3/ml-latest-small/movies.csv')
#genomescores=pd.read_csv('/home/andres/Documents/Jupyter/Enrriquecimient Semantic/ml-latest/genome-scores.csv')
ratings=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SistemasRecomendacion/Talleres/Taller3/ml-latest-small/ratings.csv',nrows= 1000)
#tags=pd.read_csv('/home/andres/Documents/Jupyter/Enrriquecimient Semantic/ml-latest/tags.csv')
#links=pd.read_csv('/home/andres/Documents/Jupyter/Enrriquecimient Semantic/ml-latest/links.csv')

In [6]:
movies.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


In [7]:
ratings.describe()

,userId,movieId,rating,timestamp
count,1000.000000,1000.00000,1000.000000,1.000000e+03
mean,4.257000,3914.68200,3.669000,9.679686e+08
std,2.148387,13928.97102,1.175509,1.381283e+08
min,1.000000,1.00000,0.500000,8.455531e+08
25%,2.000000,358.75000,3.000000,8.455555e+08
50%,4.000000,1042.00000,4.000000,9.646233e+08
75%,6.000000,2649.50000,5.000000,9.868489e+08
max,7.000000,131724.00000,5.000000,1.445715e+09


In [8]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [10]:
joined_df = pd.merge(movies, ratings, on='movieId')

In [11]:
joined_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,2,Jumanji (1995),Adventure|Children|Fantasy,6,4.0,845553522
4,3,Grumpier Old Men (1995),Comedy|Romance,1,4.0,964981247


In [12]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
from surprise import Reader
reader = Reader( rating_scale = ( 1, 5 ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings[ [ 'userId', 'movieId', 'rating' ] ], reader )

In [13]:
train_set, test_set=  train_test_split(surprise_dataset, test_size=.2)

In [14]:
train_set.n_ratings

800

In [15]:
len(test_set)

200

In [16]:
test_set[0]

(1, 804, 4.0)

In [17]:
# se crea un modelo knnbasic item-item con similitud coseno 
sim_options = {'name': 'cosine',
               'user_based': False  # calcule similitud item-item
               }
algo = KNNBasic(k=20, min_k=2, sim_options=sim_options)

In [18]:
algo.fit(trainset=train_set)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [19]:
algo.predict(154,302)

Prediction(uid=154, iid=302, r_ui=None, est=3.6525, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [20]:
rating_data=surprise_dataset.build_full_trainset()

In [21]:
#for index,row in movies.iterrows():
#  print (index, row['movieId'],row['title']) 


In [22]:
dataframelist = []

In [23]:
for uid, iid, rating in rating_data.all_ratings():
    for index,row in movies.iterrows():
      if  row['movieId'] == iid: 
       data  = {}
       data['movieId'] = row['movieId']
       data['film_title'] = row['title']
       data['rating'] = rating
       data['userId'] = uid
       data['itemId'] = iid
       dataframelist.append(data)
       #print (index, row['movieId'],row['title']) 
       #print(uid, iid, rating)


In [80]:
dataframelist

[{'movieId': 1,
  'film_title': 'Toy Story (1995)',
  'rating': 4.0,
  'userId': 0,
  'itemId': 1},
 {'movieId': 2,
  'film_title': 'Jumanji (1995)',
  'rating': 4.0,
  'userId': 0,
  'itemId': 2},
 {'movieId': 3,
  'film_title': 'Grumpier Old Men (1995)',
  'rating': 5.0,
  'userId': 0,
  'itemId': 3},
 {'movieId': 4,
  'film_title': 'Waiting to Exhale (1995)',
  'rating': 5.0,
  'userId': 0,
  'itemId': 4},
 {'movieId': 5,
  'film_title': 'Father of the Bride Part II (1995)',
  'rating': 3.0,
  'userId': 0,
  'itemId': 5},
 {'movieId': 6,
  'film_title': 'Heat (1995)',
  'rating': 5.0,
  'userId': 0,
  'itemId': 6},
 {'movieId': 7,
  'film_title': 'Sabrina (1995)',
  'rating': 4.0,
  'userId': 0,
  'itemId': 7},
 {'movieId': 8,
  'film_title': 'Tom and Huck (1995)',
  'rating': 5.0,
  'userId': 0,
  'itemId': 8},
 {'movieId': 9,
  'film_title': 'Sudden Death (1995)',
  'rating': 5.0,
  'userId': 0,
  'itemId': 9},
 {'movieId': 10,
  'film_title': 'GoldenEye (1995)',
  'rating': 5.0,


In [81]:
df_base = pd.DataFrame(dataframelist)

In [82]:
df_base.head()

,movieId,film_title,rating,userId,itemId
0,1,Toy Story (1995),4.0,0,1
1,2,Jumanji (1995),4.0,0,2
2,3,Grumpier Old Men (1995),5.0,0,3
3,4,Waiting to Exhale (1995),5.0,0,4
4,5,Father of the Bride Part II (1995),3.0,0,5


In [83]:
df_base['film_title'] = df_base['film_title'].apply(lambda x: x.split('(')[0].strip())

In [84]:
df_base.head()

,movieId,film_title,rating,userId,itemId
0,1,Toy Story,4.0,0,1
1,2,Jumanji,4.0,0,2
2,3,Grumpier Old Men,5.0,0,3
3,4,Waiting to Exhale,5.0,0,4
4,5,Father of the Bride Part II,3.0,0,5


##Modelo Ontologico

In [86]:
#!pip3 install SPARQLWrapper

In [87]:
#importing libraries
import numpy as np
import pandas as pd

from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm

In [88]:
# Specify the DBPedia endpoint
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [89]:
#specifying the query

sparql.setQuery("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX  dbpedia-owl:  <http://dbpedia.org/ontology/>
    
    SELECT ?film_title ?star_name ?nameDirector WHERE {
      {  
        SELECT DISTINCT ?movies ?film_title
        WHERE {
           ?movies rdf:type <http://dbpedia.org/ontology/Film>; 
           rdfs:label ?film_title.
        } 
      }. 
      ?movies dbpedia-owl:starring ?star;
      dbpedia-owl:director ?director. 
      ?director foaf:name ?nameDirector.
      ?star foaf:name ?star_name.
      FILTER LANGMATCHES(LANG(?film_title), 'es')
    }
""")

In [90]:
%%time

# Convert results to JSON format
sparql.setReturnFormat(JSON)
result = sparql.query().convert()

CPU times: user 86.1 ms, sys: 12 ms, total: 98.1 ms
Wall time: 6.81 s


In [91]:
result.keys()

dict_keys(['head', 'results'])

In [92]:
#attributes 
result['head']['vars']

['film_title', 'star_name', 'nameDirector']

In [93]:
attributes = list(result['results']['bindings'][0].keys())

df = pd.DataFrame(columns=attributes)
df

,film_title,star_name,nameDirector


In [94]:
n_examples = len(result['results']['bindings'])
n_examples

10000

In [95]:
for idx, example in tqdm(enumerate(result['results']['bindings']), total=n_examples):
    for attr in attributes:
        df.loc[idx,attr] = example[attr]['value']

100%|██████████| 10000/10000 [00:11<00:00, 876.97it/s]


In [96]:
df.head()

,film_title,star_name,nameDirector
0,Vanilla Sky,Cameron Diaz,Cameron Crowe
1,Viaggi di nozze,Carlo Verdone,Carlo Verdone
2,Io e mia sorella,Carlo Verdone,Carlo Verdone
3,Crucero de placer (película),Roberto Carnaghi,Carlos Borcosque Jr.
4,Río 2,Rodrigo Santoro,Carlos Saldanha


In [97]:
df.describe()

,film_title,star_name,nameDirector
count,10000,10000,10000
unique,2306,5098,724
top,Tales of Halloween,Mae Questel,Sam Raimi
freq,66,67,213


In [98]:
df.isnull().sum()

film_title      0
star_name       0
nameDirector    0
dtype: int64

In [99]:
def find_smallest_string(df,label):
    return min(map(lambda x: len(x), df[label]))

print('Smallest string - film_title: ', find_smallest_string(df,'film_title'))
print('Smallest string - star_name: ', find_smallest_string(df,'star_name'))
print('Smallest string - nameDirector: ', find_smallest_string(df,'nameDirector'))

Smallest string - film_title:  3
Smallest string - star_name:  0
Smallest string - nameDirector:  0


In [100]:
df[df['star_name']==''].head(5)

,film_title,star_name,nameDirector
75,La pasión de Juana de Arco,,Carl Theodor Dreyer
314,The Agony and the Ecstasy (película),,Carol Reed
547,Casino Royale (película de 1967),,Robert Parrish
584,Savior,,Predrag Antonijević
792,Unfaithfully Yours (1948),,Preston Sturges


In [101]:
df = df.replace('',np.nan)

In [102]:
df.isnull().sum()

film_title       0
star_name       61
nameDirector    64
dtype: int64

In [103]:
total_null_values = np.sum(df.isnull().sum()) 
total_values = df.shape[0] * df.shape[1]

missing_perc = total_null_values / total_values * 100
print(f'Percentage of missing values {missing_perc.round(2)} %')

Percentage of missing values 0.42 %


In [104]:
#df.to_csv('movie_dbpedia_final.csv',index=False)

## Hibridación

In [105]:
# df_base es el dataset de insumo para el filtro colaborativo
# df es el dataset del resultado del filtro ontologico
final_df = pd.merge(df_base, df , on='film_title', how='left')

In [131]:
# Se hace limpieza al resultado del merge porque no siempre los resultado del filtro colaborativo estan en el filtro ontologico
final_df = final_df.dropna()

In [132]:
# Se obtiene el dataset del filtro colaborativo enrriquecido por el nombre del actor y el nombre del director, obteniendo asi solo los resultados del filtro colaborativo
final_df.head()

,movieId,film_title,rating,userId,itemId,star_name,nameDirector
2,3,Grumpier Old Men,5.0,0,3,Ann-Margret,Howard Deutch
3,3,Grumpier Old Men,5.0,0,3,Walter Matthau,Howard Deutch
4,3,Grumpier Old Men,5.0,0,3,Daryl Hannah,Howard Deutch
5,3,Grumpier Old Men,5.0,0,3,Sophia Loren,Howard Deutch
20,18,Four Rooms,5.0,0,18,Sammi Davis,Quentin Tarantino


In [144]:
# Este merge es el modelo hibrido, apartir de las recomendaciones del modelo colaborativo buscamos en el filtro ontologico que actores 
# participaron en otra peliculas 
df_star_name = pd.merge( final_df, df , on='star_name').drop('nameDirector_y', axis=1)

In [178]:
df_star_name.head()

,movieId,film_title_x,rating,userId,itemId,star_name,nameDirector_x,film_title_y
0,3,Grumpier Old Men,5.0,0,3,Ann-Margret,Howard Deutch,The Cheap Detective
1,3,Grumpier Old Men,5.0,0,3,Ann-Margret,Howard Deutch,El gran robo (película de 1968)
2,3,Grumpier Old Men,5.0,0,3,Ann-Margret,Howard Deutch,Grumpier Old Men
3,3,Grumpier Old Men,2.0,3,3,Ann-Margret,Howard Deutch,The Cheap Detective
4,3,Grumpier Old Men,2.0,3,3,Ann-Margret,Howard Deutch,El gran robo (película de 1968)


In [146]:
# Este merge es el modelo hibrido, apartir de las recomendaciones del modelo colaborativo buscamos en el filtro ontologico que directores  
# participaron en otra peliculas 
df_name_director = pd.merge( final_df, df , on='nameDirector').drop('star_name_y', axis=1)

In [179]:
df_name_director.head()

,movieId,film_title_x,rating,userId,itemId,star_name_x,nameDirector,film_title_y
0,3,Grumpier Old Men,5.0,0,3,Ann-Margret,Howard Deutch,Getting Even with Dad
1,3,Grumpier Old Men,5.0,0,3,Ann-Margret,Howard Deutch,La chica de rosa
2,3,Grumpier Old Men,5.0,0,3,Ann-Margret,Howard Deutch,The Replacements (película)
3,3,Grumpier Old Men,5.0,0,3,Ann-Margret,Howard Deutch,"La extraña pareja, otra vez"
4,3,Grumpier Old Men,5.0,0,3,Ann-Margret,Howard Deutch,"La extraña pareja, otra vez"


In [172]:
# Se crean las funciones para hallar las recomendaciones por usuario para peliculas recomendadas por actor
def recomndaciones_usuario_actores(usuario):
  return df_star_name.loc[df_star_name['userId'] == usuario].sort_values('rating', ascending=False)

In [173]:
recomndaciones_usuario_actores(0).head()

,movieId,film_title_x,rating,userId,itemId,star_name,nameDirector_x,film_title_y
0,3,Grumpier Old Men,5.0,0,3,Ann-Margret,Howard Deutch,The Cheap Detective
403,70,From Dusk Till Dawn,5.0,0,70,Salma Hayek,Robert Rodriguez,From Dusk Till Dawn (franquicia)
405,70,From Dusk Till Dawn,5.0,0,70,Salma Hayek,Robert Rodriguez,Grown Ups
406,70,From Dusk Till Dawn,5.0,0,70,Salma Hayek,Robert Rodriguez,Grown Ups 2
407,70,From Dusk Till Dawn,5.0,0,70,Salma Hayek,Robert Rodriguez,El callejón de los milagros (película)


In [174]:
# Se crean las funciones para hallar las recomendaciones por usuario para peliculas recomendadas por director
def recomndaciones_usuario_directores(usuario):
  return df_name_director.loc[df_name_director['userId'] == usuario].sort_values('rating', ascending=False)

In [175]:
recomndaciones_usuario_directores(0).head()

,movieId,film_title_x,rating,userId,itemId,star_name_x,nameDirector,film_title_y
0,3,Grumpier Old Men,5.0,0,3,Ann-Margret,Howard Deutch,Getting Even with Dad
5997,18,Four Rooms,5.0,0,18,Jennifer Beals,Robert Rodriguez,The Faculty
5986,18,Four Rooms,5.0,0,18,Jennifer Beals,Robert Rodriguez,The Faculty
5987,18,Four Rooms,5.0,0,18,Jennifer Beals,Robert Rodriguez,The Faculty
5988,18,Four Rooms,5.0,0,18,Jennifer Beals,Robert Rodriguez,Grindhouse
